<a href="https://colab.research.google.com/github/helenatong/oc_p5_Segmentez_des_clients_d_un_site_e-commerce/blob/main/exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Introduction**

---

## **Prérequis**
- Télécharger [ce fichier TO MOD](https://drive.google.com/file/d/1NWEnq2dD_kOGe-tnFS2w1-C7apQ1ejWx/view?usp=drive_link) dans Google Drive
- Modifier les chemins du fichier (extract_path) dans le code


---

 ## **Contexte et description du notebook**

---

## **Sommaire**

### **I. Installation et mise en place de l'environnement de travail**

### **II. Définitions des fonctions et des variables globales**

### **III. Analyse exploratoire**

# **I. Installation et mise en place de l'environnement de travail**

In [1]:
# importation des librairies
import numpy as np
import pandas as pd
import sqlite3

import zipfile
import os

import re

import sys

import missingno as msno
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.impute import KNNImputer

import datetime

In [2]:
# informations sur l'environnement virtuel
print('Version des librairies :')
print('Python        : ' + sys.version)
print('NumPy         : ' + np.version.full_version)
print('Pandas        : ' + pd.__version__)
print('Matplotlib    : ' + mpl.__version__)
print('Seaborn       : ' + sns.__version__)

now  = datetime.datetime.now().isoformat()
print('Run date      : ' + now)

Version des librairies :
Python        : 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]
NumPy         : 1.25.2
Pandas        : 2.0.3
Matplotlib    : 3.7.1
Seaborn       : 0.13.1
Run date      : 2024-06-15T06:20:48.719818


In [3]:
# configurations de l'affichages des donnees
# pd.set_option('display.max_columns', None)

In [4]:
# connexion au drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# connecter la base de données
conn = sqlite3.connect('/content/drive/MyDrive/openclassrooms/project_list/project_5/data/olist.db')

tables = conn.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()
table_names = [table[0] for table in tables]

dfs = {}

for table_name in table_names:
    cursor = conn.execute(f"SELECT * FROM {table_name};")
    rows = cursor.fetchall()
    column_names = [description[0] for description in cursor.description]
    dfs[table_name] = pd.DataFrame(rows, columns=column_names).set_index('index')

    conn.commit()

for table_name, df in dfs.items():
  print(f"Table: {table_name}")

Table: customers
Table: geoloc
Table: order_items
Table: order_pymts
Table: order_reviews
Table: orders
Table: products
Table: sellers
Table: translation


In [27]:
# récupérer les dataframes
for table_name in dfs.keys():
  locals()[f'df_{table_name}'] = dfs.get(table_name)

# vérifier l'importation
for table_name in dfs.keys():
  df = locals()[f'df_{table_name}']
  print(table_name, '\n', df.head(2), end='\n\n\n')

customers 
                             customer_id                customer_unique_id  \
index                                                                       
0      06b8999e2fba1a1fbc88172c00ba8bc7  861eff4711a542e4b93843c6dd7febb0   
1      18955e83d337fd6b2def6b18a428ac77  290c77bc529b7ac935b93aa66c333dc3   

       customer_zip_code_prefix          customer_city customer_state  
index                                                                  
0                         14409                 franca             SP  
1                          9790  sao bernardo do campo             SP  


geoloc 
        geolocation_zip_code_prefix  geolocation_lat  geolocation_lng  \
index                                                                  
0                             1037       -23.545621       -46.639292   
1                             1046       -23.546081       -46.644820   

      geolocation_city geolocation_state  
index                                     
0    

# **II. Définitions des fonctions et des variables globales**
